In [18]:
import pandas as pd
import numpy as np
import re
import time
import json

import requests as rq
import bs4 as bs4

import tqdm
import glob

## Collect

In [28]:
df = pd.read_json("parsed_videos.json",lines=True)
df.head()

,link,title,query
0,/watch?v=9v_Sh6hT0W0,Arduino with Python | Python machine learning ...,machine+learning
1,/watch?v=lRs_xaeoVIk,Vehicular Damage Detection Application using M...,machine+learning
2,/watch?v=87kLfzmYBy8,DeepMind x UCL | Deep Learning Lectures | 6/12...,machine+learning
3,/watch?v=FBggC-XVF4M,DeepMind x UCL | Deep Learning Lectures | 2/12...,machine+learning
4,/watch?v=7R52wiUgxZI,DeepMind x UCL | Deep Learning Lectures | 1/12...,machine+learning


In [29]:
df['link']


0       /watch?v=9v_Sh6hT0W0
1       /watch?v=lRs_xaeoVIk
2       /watch?v=87kLfzmYBy8
3       /watch?v=FBggC-XVF4M
4       /watch?v=7R52wiUgxZI
                ...         
1374    /watch?v=X6orAXDIrds
1375    /watch?v=kBR0EtGOkzc
1376    /watch?v=ElDWanLOc4c
1377    /watch?v=xrhPjE7wHas
1378    /watch?v=i92VI289zWw
Name: link, Length: 1379, dtype: object

In [30]:
links_list = df['link'].unique()

In [31]:
len(links_list)

1358

In [32]:
url = "https://www.youtube.com{link}"

for link in links_list:
    urll = url.format(link=link)
    print(urll)
    response = rq.get(urll)
    
    link_name = re.search("v=(.*)",link).group(1)

    with open("./dados_brutos/video_{}.html".format(link_name),'w+',encoding="utf-8") as output:
        output.write(response.text)
    time.sleep(2)

https://www.youtube.com/watch?v=9v_Sh6hT0W0
https://www.youtube.com/watch?v=lRs_xaeoVIk
https://www.youtube.com/watch?v=87kLfzmYBy8
https://www.youtube.com/watch?v=FBggC-XVF4M
https://www.youtube.com/watch?v=7R52wiUgxZI
https://www.youtube.com/watch?v=kVU8zTI-Od0
https://www.youtube.com/watch?v=_hJo4v5ju-0
https://www.youtube.com/watch?v=Af9tFHNe9xY
https://www.youtube.com/watch?v=6ST-equDpuQ
https://www.youtube.com/watch?v=maeOjme3n-w
https://www.youtube.com/watch?v=yOF5VABOia0
https://www.youtube.com/watch?v=2cCAaa0RQFM
https://www.youtube.com/watch?v=vXGqaqwy-So
https://www.youtube.com/watch?v=1Q81cZAnfIA
https://www.youtube.com/watch?v=z0s43be3Des
https://www.youtube.com/watch?v=GhTREKMYp34
https://www.youtube.com/watch?v=U2d7qv3prME
https://www.youtube.com/watch?v=M0Juo-oSn4A
https://www.youtube.com/watch?v=hMgrCzZD2ew
https://www.youtube.com/watch?v=rInz-CEa-3g
https://www.youtube.com/watch?v=VeikLmU4bB8
https://www.youtube.com/watch?v=7_ANYxgQbtI
https://www.youtube.com/watch?v=

## Data processing

In [34]:
with open("parsed_video_info.json",'w+') as output:
    for video_file in tqdm.notebook.tqdm(sorted(glob.glob("./dados_brutos/video*"))):
        with open(video_file,'r+',encoding='utf-8') as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html,'html.parser')
            
            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
            channel = parsed.find_all("a",attrs={"href":re.compile(r"channel")})
            meta = parsed.find_all("meta")
            
            data = dict()
            
            for e in class_watch:
                colname = "_".join(e['class'])
                if "clearfix" in colname:
                    continue
                data[colname] = e.text.strip()
                
            for e in id_watch:
                colname = e['id']
                data[colname] = e.text.strip()
                                
            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']
                
            for link_num, e in enumerate(channel):
                data["channel_link_{}".format(link_num)] = e['href']
                
            output.write("{}\n".format(json.dumps(data)))
                

In [35]:
df = pd.read_json("parsed_video_info.json",lines=True)
df.shape

(1358, 138)

In [36]:
pd.set_option("display.max_columns",500)
df.head(1)

,content-alignment_watch-small,watch-playlist_player-height,watch-queue-header,watch-queue-info,watch-queue-info-icon,watch-queue-title,watch-queue-control-bar_control-bar-button,watch-queue-mole-info,watch-queue-control-bar-icon,watch-queue-icon_yt-sprite,watch-queue-title-container,watch-queue-count,watch-queue-menu_yt-uix-button-menu_yt-uix-button-menu-dark-overflow-action-menu_hid,watch-queue-menu-choice_overflow-menu-choice_yt-uix-button-menu-item,watch-queue-controls,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_prev-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-prev_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_play-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-play_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_pause-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_hid_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-pause_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_next-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-next_yt-sprite,watch-queue-items-container_yt-scrollbar-dark_yt-scrollbar,watch-queue-items-list,content-alignment_watch-player-playlist,watch-main-col,watch-title-container,watch-title,watch-secondary-actions_yt-uix-button-group,watch-view-count,watch-action-panels_yt-uix-button-panel_hid_yt-card_yt-card-has-padding,watch-time-text,watch-extras-section,watch-meta-item_yt-uix-expander-body,content_watch-info-tag-list,watch-sidebar,watch-playlist_player-height_hid,watch-sidebar-gutter_yt-card_yt-card-has-padding_yt-uix-expander_yt-uix-expander-collapsed,watch-sidebar-section,watch-sidebar-head,watch-sidebar-body,watch-sidebar-separation-line,yt-pl-watch-queue-overlay,watch-queue-mole,watch-queue,watch-queue-title-msg,watch-queue-count-msg,watch-queue-loading-template,watch7-container,watch7-main-container,watch7-main,watch7-preview,watch7-content,watch-header,watch7-headline,watch-headline-title,watch7-user-header,watch7-subscription-container,watch8-action-buttons,watch8-secondary-actions,watch8-sentiment-actions,watch7-views-info,watch-action-panels,watch-actions-share-loading,watch-actions-share-panel,watch-actions-rental-required,watch-description,watch-description-content,watch-description-clip,watch-uploader-info,watch-description-text,watch-description-extras,watch-discussion,watch7-sidebar,watch7-sidebar-contents,watch7-sidebar-offer,watch7-sidebar-ads,watch7-sidebar-modules,watch-related,shared-addto-watch-later-login,og:site_name,og:url,og:title,og:image,og:image:width,og:image:height,og:description,al:ios:app_store_id,al:ios:app_name,al:ios:url,al:android:url,al:android:app_name,al:android:package,al:web:url,og:type,og:video:url,og:video:secure_url,og:video:type,og:video:width,og:video:height,og:video:tag,fb:app_id,channel_link_0,channel_link_1,watch-actions-transcript-loading,watch-actions-transcript,watch-transcript-container,watch-transcript-not-found,watch-sidebar-live-chat,channel_link_2,channel_link_3,channel_link_4,channel_link_5,channel_link_6,channel_link_7,channel_link_8,channel_link_9,channel_link_10,channel_link_11,channel_link_12,watch-channel-brand-div,watch-channel-brand-div-text,watch-sidebar-discussion,watch-skeleton,watch-page-skeleton,watch-meta-item,channel_link_13,watch-meta-item_has-image,channel_link_14,channel_link_15,channel_link_16,channel_link_17,channel_link_18,channel_link_19,channel_link_20,channel_link_21,channel_link_22
0,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fi

In [37]:
selected_columns = ['watch-title', 'watch-view-count', 'watch-time-text', 'content_watch-info-tag-list', 'watch7-headline',
                    'watch7-user-header', 'watch8-sentiment-actions', "og:image", 'og:image:width', 'og:image:height',
                    "og:description", "og:video:width", 'og:video:height', "og:video:tag", 'channel_link_0']

In [38]:
df[selected_columns].head()

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,Python For Data Science Full Course - 9 Hours ...,33.445 visualizações,Publicado em 15 de mar. de 2020,Educação,#edureka #PythonEdureka #pythonfordatasciencef...,edureka!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregan...,33.445 visualizações\n\n\n\n\n\n\n\n1.003\n\nG...,https://i.ytimg.com/vi/-6RqxhNO2yY/maxresdefau...,1280,720,🔥Edureka Python Certification Training: https:...,1280.0,720.0,edureka,/channel/UCkw4JCwteGrDHIsyIIKo4tQ
1,Michael I. Jordan: Machine Learning: Dynamical...,4.640 visualizações,Publicado em 2 de mai. de 2019,Licença de atribuição Creative Commons (reutil...,#purdue #michaelijordan #engineering\n\n\n\n ...,Purdue Engineering\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,4.640 visualizações\n\n\n\n\n\n\n\n98\n\nGosto...,https://i.ytimg.com/vi/-8yYFdV5SOc/maxresdefau...,1280,720,2019 Purdue Engineering Distinguished Lecture ...,1280.0,720.0,electrical engineer,/channel/UC8FZ6dzFVkCACLH9YoMNFog
2,Kaggle Live-Coding: RNNs for Sarcasm Detection...,1.570 visualizações,Transmitido ao vivo em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Live-Coding: RNNs for Sarcasm Detection...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.570 visualizações\n\n\n\n\n\n\n\n34\n\nGosto...,https://i.ytimg.com/vi/-9U84J178OQ/maxresdefau...,1280,720,Join Kaggle data scientist Rachael live as she...,1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA
3,Can You Become a Data Scientist?,699.800 visualizações,Publicado em 14 de ago. de 2018,Educação,#data #science #datascientist\n\n\n\n Can Y...,365 Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\nCa...,699.800 visualizações\n\n\n\n\n\n\n\n10.749\n\...,https://i.ytimg.com/vi/-AkBfBWr_Gw/maxresdefau...,1280,720,👉 Download Our Free Data Science Career Guide:...,1280.0,720.0,data scientist,/channel/UCEBpSZhI1X8WaP-kY_2LLcg
4,Binary Image Classification with CNN (Deep Lea...,300 visualizações,Publicado em 23 de mar. de 2020,Ciência e tecnologia,#download #Kaggle #Google\n\n\n\n Binary Im...,TheClassofAI\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,300 visualizações\n\n\n\n\n\n\n\n9\n\nGostou d...,https://i.ytimg.com/vi/-DDrE6T0ct4/hqdefault.jpg,480,360,"In this tutorial, we have covered how to train...",1280.0,720.0,ImageClassification,/channel/UCYlOdJBJQN4c7k25uzwSwJA


In [39]:
df[selected_columns].to_feather("raw_data.feather")

In [40]:
df[selected_columns].to_csv("raw_data_without_labels.csv")